In [35]:
from __future__ import print_function
import numpy as np
import os
import subprocess
import sys
import re
import random
import pprint as pp
from tabulate import tabulate

In [36]:
def clean(matrix, tol=1e-10):
    arr = matrix.copy()

    # Flush small elements to zero
    arr[np.abs(arr) < tol] = 0

    # Coalesce similar elements
    if arr.size > 1:
        amfl = np.abs(arr.flat)
        amix = np.argsort(amfl)

        i, ix = 0, amix[0]
        for j, jx in enumerate(amix[1:], start=1):
            if amfl[jx] - amfl[ix] >= tol:
                if j - i > 1:
                    amfl[amix[i:j]] = np.median(amfl[amix[i:j]])
                i, ix = j, jx

        if i != j:
            amfl[amix[i:]] = np.median(amfl[amix[i:]])

        # Fix up the signs and assign
        arr.flat = np.copysign(amfl, arr.flat)

    return arr

def count_unique_constants(matrix, tol=1e-10):
    arr = matrix.copy()
    arr = arr[abs(arr) >= tol]
    unique, counts = np.unique(arr, return_counts=True)
    return np.array((unique, counts)).T

def num_unique_constants(matrix):
    return len(count_unique_constants(matrix))

In [37]:
def gen_matrix_kernels(file_name):
    clean_mat  = clean(np.loadtxt(file_name))
    rows       = np.size(clean_mat, 0)
    columns    = np.size(clean_mat, 1)
    density    = float(np.count_nonzero(clean_mat)) / float(clean_mat.size)
    num_unique = num_unique_constants(clean_mat)
    
    return rows, columns, density, num_unique

def atoi(text):
    return int(text) if text.isdigit() else text
def natural_keys(text):
    return [atoi(c) for c in re.split(r'(\d+)', text)]


In [87]:
mats_dir = "./../example_mats/tet/p1/shunn-ham"
mat_paths = sum([[os.path.join(dir, file) for file in files] for dir, _, files in os.walk(mats_dir)], [])
mat_paths.sort(key=natural_keys)

headers = ["Matrix", "$R$", "$C$", "$p$", "$U$"]
data = []
if mat_paths:
    for mat_path in mat_paths:
        m = [ mat_path.split('/')[-1].split('.txt')[0] ] # mX
        characteristics = gen_matrix_kernels(mat_path)
        m.extend(characteristics)
        data.append(m)
        table = tabulate(data, headers, tablefmt="latex_raw", floatfmt=".4f")
    print(table)

\begin{tabular}{lrrrr}
\hline
 Matrix   &   $R$ &   $C$ &    $p$ &   $U$ \\
\hline
 m0       &    12 &     4 & 1.0000 &     3 \\
 m3       &     4 &    12 & 1.0000 &     6 \\
 m6       &    12 &    12 & 0.5000 &     6 \\
 m132     &     4 &    12 & 0.5000 &     2 \\
 m460     &    12 &     4 & 0.5000 &     2 \\
\hline
\end{tabular}
